In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [4]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [6]:
import pandas as pd
from datasets import Dataset

# Load dataset
file_path = '/content/drive/MyDrive/ML_project/ML_project/merged_data.csv'
df = pd.read_csv(file_path)

# Rename columns to match T5 input-output format
df = df.rename(columns={'sentence': 'input', 'corrections': 'target'})

# Add a prefix to the input column to guide the T5 model
df['input'] = "fix grammar: " + df['input']

# Drop any unnecessary columns (if present)
df = df[['input', 'target']]

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer and model for Grammarly CoEdit
tokenizer = AutoTokenizer.from_pretrained('grammarly/coedit-large')
model = AutoModelForSeq2SeqLM.from_pretrained('grammarly/coedit-large')

# Tokenization function for T5
def tokenize_function(examples):
    # Tokenize the input and target text
    model_inputs = tokenizer(
        examples['input'],
        max_length=64,
        truncation=True,
        padding='max_length'
    )

    # Tokenize target sentences (labels)
    labels = tokenizer(
        examples['target'],
        max_length=64,
        truncation=True,
        padding='max_length'
    )

    # Add labels to the model inputs
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply tokenization to the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['input', 'target'])


tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

Map:   0%|          | 0/2772 [00:00<?, ? examples/s]

In [8]:
print(tokenized_dataset.features)

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}


In [9]:
from transformers import TrainingArguments, Trainer, AutoModelForSeq2SeqLM
from transformers import Trainer

# Load the Grammarly CoEdit model
model = AutoModelForSeq2SeqLM.from_pretrained('grammarly/coedit-large')

# Define Training Arguments
training_args = TrainingArguments(
    output_dir='./results',           # Save model and checkpoints
    num_train_epochs=15,              # More epochs for small datasets
    per_device_train_batch_size=8,    # Slightly larger batch size for stability
    per_device_eval_batch_size=8,     # Same as train batch size
    warmup_steps=500,                 # More warmup steps for smoother training
    weight_decay=0.01,                # Regularization to prevent overfitting
    logging_dir='./logs',             # Log directory
    logging_steps=10,                 # Log metrics every 10 steps
    evaluation_strategy="epoch",      # Evaluate after every epoch
    save_strategy="epoch",            # Save checkpoints after every epoch
    save_total_limit=2,               # Keep only the last 2 checkpoints
    learning_rate=2e-5,               # Slightly lower learning rate
    load_best_model_at_end=True,      # Automatically load the best model
    metric_for_best_model="loss",     # Optimize for validation loss
    greater_is_better=False,          # Lower loss is better
    # predict_with_generate=True,     # Removed from TrainingArguments
    fp16=True,                        # Enable mixed precision training for speed (if supported by GPU)
    report_to="none"                  # Disable Weights & Biases integration
)


# Define the Trainer
trainer = Trainer(
    model=model,                       # The pre-trained BERT model
    args=training_args,                # Training arguments (defined earlier)
    train_dataset=tokenized_dataset,   # Tokenized dataset for training
    eval_dataset=tokenized_dataset,    # Use the same dataset for evaluation (if no separate eval dataset)
    tokenizer=tokenizer,               # Tokenizer used for preprocessing
    compute_metrics=None               # Optional: Define custom metrics if needed
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-c6c8f0cab578>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.000000,nan
2,0.000000,nan
3,0.000000,nan
4,0.000000,nan
5,0.000000,nan
6,0.000000,nan
7,0.000000,nan
8,0.000000,nan
9,0.000000,nan
10,0.000000,nan


TrainOutput(global_step=5205, training_loss=0.0, metrics={'train_runtime': 3362.2576, 'train_samples_per_second': 12.367, 'train_steps_per_second': 1.548, 'total_flos': 1.197857362673664e+16, 'train_loss': 0.0, 'epoch': 15.0})

In [11]:
import torch

# Correct a sentence using the fine-tuned model
def correct_sentence(sentence):
    input_text = f"fix grammar: {sentence}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=64, truncation=True).to('cuda' if torch.cuda.is_available() else 'cpu')

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=64, num_beams=5, early_stopping=True)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test Example
test_sentence = "I is happy to saw you."
corrected = correct_sentence(test_sentence)
print("Original Sentence:", test_sentence)
print("Corrected Sentence:", corrected)


Original Sentence: I is happy to saw you.
Corrected Sentence: I am happy to see you.


In [12]:
model.save_pretrained('/content/drive/MyDrive/ML_project/coedit')
tokenizer.save_pretrained('/content/drive/MyDrive/ML_project/coedit')

('/content/drive/MyDrive/ML_project/coedit/tokenizer_config.json',
 '/content/drive/MyDrive/ML_project/coedit/special_tokens_map.json',
 '/content/drive/MyDrive/ML_project/coedit/spiece.model',
 '/content/drive/MyDrive/ML_project/coedit/added_tokens.json',
 '/content/drive/MyDrive/ML_project/coedit/tokenizer.json')